In [1]:
from multiprocessing import Process, Lock
from multiprocessing.sharedctypes import Array

import numpy as np

In [2]:
def mod_list(index, shared_arr):
    shared_arr[index] = index

In [3]:
lock = Lock()
arr = Array('i', 10, lock=lock)
for i in range(0, 10):
    p = Process(target=mod_list, args=(i, arr))
    p.start()
    p.join()

X = list(arr)
X

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Single Thread Version
https://jonasteuwen.github.io/numpy/python/multiprocessing/2017/01/07/multiprocessing-numpy-array.html

In [4]:
import itertools

In [5]:
# Example 2
X = np.random.random((100, 100))
tmp = np.zeros((100, 100))

window_idxs = [(i, j) for i, j in itertools.product(range(0, 100, 2), range(0, 100, 2))]

In [6]:
def fill_per_window(args):
    window_x, window_y = args
    
    for i in range(window_x, window_y+2):
        for j in range(window_x, window_y+2):
            tmp[i, j] = X[i, j]

for idx in window_idxs:
    fill_per_window(idx)
    
print(np.array_equal(X, tmp))

True


In [7]:
window_idxs[:5]

[(0, 0), (0, 2), (0, 4), (0, 6), (0, 8)]

## Multithread version

In [8]:
from multiprocessing import sharedctypes, Pool
from threading import currentThread

In [9]:
# Multiprocessing version
size = 100
block_size = 4

result = np.ctypeslib.as_ctypes(np.zeros((size, size)))
shared_arr = sharedctypes.RawArray(result._type_, result)

In [10]:
def fill_per_window(args):
#     print(currentThread())
    window_x, window_y = args
    tmp = np.ctypeslib.as_array(shared_arr)

    for idx_x in range(window_x, window_x + block_size):
        for idx_y in range(window_y, window_y + block_size):
            tmp[idx_x, idx_y] = X[idx_x, idx_y]


In [11]:
window_idxs = [(i, j) for i, j in
               itertools.product(range(0, size, block_size),
                                 range(0, size, block_size))]

In [12]:
with Pool(processes=8) as p:
    res = p.map(fill_per_window, window_idxs)
    result = np.ctypeslib.as_array(shared_arr)

print(np.array_equal(X, result))

True


array([[0.34117379, 0.99301816, 0.00426082, ..., 0.88827464, 0.94146552,
        0.74393458],
       [0.14914119, 0.49264682, 0.80953164, ..., 0.0368246 , 0.70316185,
        0.05739841],
       [0.14839939, 0.4209693 , 0.83370336, ..., 0.02118329, 0.21497169,
        0.59779232],
       ...,
       [0.56782366, 0.93136392, 0.75894413, ..., 0.08589315, 0.39620322,
        0.7315583 ],
       [0.11674814, 0.78404617, 0.83884989, ..., 0.87436799, 0.29284751,
        0.36018056],
       [0.03643574, 0.64823418, 0.34528263, ..., 0.06989658, 0.45551379,
        0.63960549]])